In [0]:
%run "../include/configuration"

In [0]:
from pyspark.sql.types import *

In [0]:
test_schema = StructType(fields=[StructField("country", StringType(), False),
                                     StructField("country_code", StringType(), False),
                                     StructField("year_week", StringType(), False),
                                     StructField("level", StringType(), True),
                                     StructField("region", StringType(), True),
                                     StructField("region_name", StringType(), True),
                                     StructField("new_cases", IntegerType(), True),
                                     StructField("tests_done", IntegerType(), True),
                                     StructField("population", IntegerType(), True),
                                     StructField("testing_rate", DoubleType(), True),
                                     StructField("positivity_rate", DoubleType(), True),
                                     StructField("testing_data_source", StringType(), True)])

In [0]:
test_df = spark.read.option("header", True).schema(test_schema).csv(f"{raw_covid_folder_path}/test.csv")

In [0]:
lookup_date_df = spark.read.option("header", True).csv(f"{raw_covid_folder_path}/lookup/dim_date.csv")

In [0]:
from pyspark.sql.functions import *

In [0]:
year_week_lookup_df = lookup_date_df.withColumn("year_week_1", regexp_replace(col("year_week"), "(\\d{4})(\\d{2})", "$1-W$2"))

In [0]:
year_week_lookup_df = year_week_lookup_df.groupBy("year_week_1").agg(min(col("date")).alias("week_first_date"), max(col("date")).alias("week_last_date"))

In [0]:
joined_test_df = test_df.join(year_week_lookup_df, test_df.year_week == year_week_lookup_df.year_week_1)

In [0]:
final_test_df = joined_test_df.select(col("year_week"), col("week_first_date"), col("week_last_date"), col("country"), col("country_code").alias("country_code_2_digit"), col("population"), col("new_cases"), col("tests_done"), col("testing_rate"), col("positivity_rate"), col("testing_data_source")).orderBy(desc("year_week"))

In [0]:
display(final_test_df)

year_week,week_first_date,week_last_date,country,country_code_2_digit,population,new_cases,tests_done,testing_rate,positivity_rate,testing_data_source
2022-W52,2022-12-18,2022-12-24,Austria,AT,8978929,24803,null,null,null,NA
2022-W52,2022-12-18,2022-12-24,Belgium,BE,11617623,6078,388,3.3397537516925793,null,TESSy COVID-19
2022-W52,2022-12-18,2022-12-24,Bulgaria,BG,6838937,877,10751,157.2027933580906,8.157380708771278,TESSy COVID-19
2022-W52,2022-12-18,2022-12-24,Croatia,HR,3862305,5207,34483,892.8088278890456,15.10019429863991,TESSy COVID-19
2022-W52,2022-12-18,2022-12-24,Cyprus,CY,904705,2756,53938,5961.943395913585,5.109570247320998,TESSy COVID-19
2022-W52,2022-12-18,2022-12-24,Czechia,CZ,10516707,2699,26369,250.73437911696124,10.235503811293565,TESSy COVID-19
2022-W52,2022-12-18,2022-12-24,Denmark,DK,5873420,6792,41663,709.348216201123,16.302234596644503,TESSy COVID-19
2022-W52,2022-12-18,2022-12-24,Estonia,EE,1331796,541,14639,1099.1923688012278,3.6956076234715485,TESSy COVID-19
2022-W52,2022-12-18,2022-12-24,Finland,FI,5548241,4246,21730,391.6556616772775,19.539806718821904,TESSy COVID-19
2022-W52,2022-12-18,2022-12-24,France,FR,67871925,154693,790720,1165.0177890195394,19.563562322946176,TESSy COVID-19


In [0]:
final_test_df.write.mode("overwrite").parquet(f"{processed_covid_folder_path}/tests")

In [0]:
test_df = spark.read.option("header", True).parquet(f"{processed_covid_folder_path}/tests")

In [0]:
%sql
USE covid_19

In [0]:
%sql
DROP TABLE covid_19.tests

In [0]:
test_df.write.format("parquet").saveAsTable("tests")